$$ Pr(sentence) = Pr(w_1 \cdot w_2 \cdots w_n) = \prod \frac{count(w_i, w_{i+1})}{count(w_i)}$$

## 导入数据

In [3]:
corpus = '/Users/Lenovo/Desktop/Data/train.txt'

In [6]:
FILE = open(corpus.strip(), encoding='UTF-8').read()

## 导入jieba分词库

In [11]:
import jieba
from collections import Counter

## 对文本进行切分分词

In [12]:
def cut():
    word_1 = FILE.replace('++$++','')
    word_2 = word_1.replace(' ','')
    word_3 = word_2.replace('\n','')
    return list(jieba.cut(word_3))

## 生成2-gram模型

In [16]:
TOKENS = cut()
words_count = Counter(TOKENS)
_2_gram_words = [TOKENS[i] + TOKENS[i+1] for i in range(len(TOKENS)- 1)  ]
_2_gram_word_counts = Counter(_2_gram_words)

In [18]:
words_count.most_common(20)

[('-', 13317),
 ('?', 12876),
 ('？', 12736),
 ('insurance', 11158),
 ('保险', 5013),
 ('的', 3220),
 ('人寿保险', 2962),
 ('什么', 2677),
 ('吗', 2479),
 ('是', 2344),
 ('我', 2053),
 ('是否', 1862),
 ('可以', 1704),
 ('健康', 1513),
 ('如何', 1294),
 ('医疗保险', 1269),
 ('多少', 1252),
 ('汽车保险', 1189),
 ('在', 913),
 ('覆盖', 848)]

## 客户语言定义

In [19]:
user = """
user = 自己 时间 地点 事件
自己 = 我 | 俺 | 我们 
时间 = 刚刚 | 今天 | 一小时前
地点 = 在路上 | 在医院 
事件 = 出车祸 | 做手术 | 摔跤
"""

## 客服语言定义

In [20]:
host = """
host = 寒暄 报数 询问 业务相关 结尾 
报数 = 我是 数字 号客服 ,
数字 = 单个数字 | 数字 单个数字 
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
寒暄 = 称谓 打招呼 | 打招呼
称谓 = 人称 ,
人称 = 先生 | 女士 | 小朋友
打招呼 = 你好 | 您好 
询问 = 请问你要 | 您需要
业务相关 = 投保 具体业务
投保 = 购买 | 咨询 | 退订
具体业务 = 报案 | 意外保险 | 汽车保险 | 医疗保险
结尾 = 吗？"""

## 清洗数据

In [21]:
import random
def get_generation_by_gram(grammar_str, target, stmt_split='=', or_split='|'):
    rules = dict()  #数据用字典存储，对象为字典
    for line in grammar_str.split('\n'):
        if not line: continue
#         使单词分开
        stmt, expression = line.split(stmt_split)   
#         去除单词后的空格
        rules[stmt.strip()] = expression.split(or_split)
        generated = generate (rules, target = target)
    return generated   

## 依据语法生成一句话

In [22]:
def generate(grammar_rule, target):
    if target in grammar_rule: 
#         取得对应的数据集grammer_rule
        words = grammar_rule[target] 
#     依据语法随机生成一句话
        word = random.choice(words) 
        return ''.join(generate(grammar_rule, target=c.strip()) for c in word.split()) #去掉首尾空格
    else:
        return target

In [27]:
# 语句生成测试
for i in range(5):
    print(get_generation_by_gram(host,target='host')) 
    print(get_generation_by_gram(user,target='user'))
     

你好我是7593号客服,您需要购买报案吗？
我们今天在路上做手术
先生,你好我是375号客服,请问你要咨询汽车险吗？
我今天在医院做手术
你好我是4638号客服,您需要咨询意外险吗？
俺今天在路上做手术
女士,您好我是2号客服,您需要购买报案吗？
俺今天在医院摔跤
您好我是7542号客服,请问你要咨询意外险吗？
我们今天在医院摔跤


## gram模型实现

In [28]:
def get_1_gram_count(word):
    if word in words_count: return words_count[word]
    else:
        return words_count.most_common()[-1][-1]

In [30]:
def get_2_gram_count(word):
    if word in _2_gram_word_counts: return _2_gram_word_counts[word]
    else:
        return _2_gram_word_counts.most_common()[-1][-1]

In [31]:
def get_gram_count(word, wc):
    if word in wc: return wc[word]
    else:
        return wc.most_common()[-1][-1]

In [35]:
def two_gram_model(sentence):
    # 2-gram langauge model
    tokens = list(jieba.cut(sentence))
    
    probability = 1
    
    for i in range(len(tokens)-1):
        word = tokens[i]
        next_word = tokens[i+1]
        
        _two_gram_c = get_gram_count(word+next_word, _2_gram_word_counts)
        _one_gram_c = get_gram_count(next_word, words_count)
        pro =  _two_gram_c / _one_gram_c
        
        probability *= pro
    
    return probability  

In [37]:
two_gram_model('怎么赔偿人寿保险')

1.776893279789616e-05

In [60]:
def generate_say(grammar_str: str, target, stmt_split='=', or_split='|'):
    rules = dict()
    for line in grammar_str.split('\n'):
        if not line:continue
        stmt, expr = line.split(stmt_split)
        rules[stmt.strip()] = expr.split(or_split)
    generated = generate(rules, target=target)
    return generated

In [87]:
def generate_n(grammar_str: str, target, n : int):
    sentences = list()
    for i in range(n):
        sentences.append(generate_say(grammar_str, target))
    return sentences

In [90]:
def generate_best(grammar_str: str, target: str, n : int):
    sentance = list()
    sentance_list = generate_n(grammar_str,target,5)
    for i in sentance_list:
        sentance.append([i, two_gram_model(i)])
    sentance_sort_list = sorted(sentance,key=lambda x: x[1], reverse=True)
    
    return sentance_sort_list[0]

In [94]:
for i in range(10):
    print(generate_best(host,'host',5))
    print(generate_best(user,'user',4))
    print()

['你好我是39号客服,您需要咨询报案吗？', 1.148284360096569e-16]
['我刚刚在路上出车祸', 0.0007301935012778385]

['小朋友,您好我是96号客服,请问你要咨询报案吗？', 6.579121050569782e-19]
['我们今天在路上出车祸', 0.0007301935012778385]

['您好我是4582号客服,请问你要退订医疗险吗？', 2.9606044727564026e-17]
['俺今天在路上出车祸', 0.0007301935012778385]

['女士,您好我是83号客服,您需要退订医疗险吗？', 2.55174302243682e-18]
['我刚刚在路上做手术', 0.0007301935012778385]

['你好我是4号客服,您需要咨询报案吗？', 5.741421800482845e-17]
['我们今天在路上做手术', 0.0007301935012778385]

['小朋友,你好我是5号客服,请问你要退订医疗险吗？', 6.579121050569782e-19]
['我刚刚在路上摔跤', 0.0007301935012778385]

['您好我是5号客服,您需要退订意外险吗？', 1.148284360096569e-16]
['我刚刚在路上做手术', 0.0007301935012778385]

['您好我是775号客服,您需要退订意外险吗？', 1.148284360096569e-16]
['我刚刚在医院出车祸', 6.845564074479737e-05]

['女士,您好我是5号客服,请问你要退订医疗险吗？', 6.579121050569782e-19]
['我刚刚在路上出车祸', 0.0007301935012778385]

['你好我是1号客服,请问你要退订报案吗？', 2.6914586115967292e-18]
['我今天在路上出车祸', 0.0007301935012778385]

